In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import h5py
import copy
import datetime
import ta
#import yfinance as yf
# import tensorflow as tf
# import tensorflow.compat.v2.feature_column as fc
from IPython.display import clear_output
import pyodbc

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, roc_auc_score, f1_score, log_loss

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.exceptions import ConvergenceWarning 
from sklearn import ensemble
# ConvergenceWarning('ignore')
# Do you wanna see?
verbose = True

import sys
sys.path.append('../')

from utils.data_extraction import load_data_final,load_data_and_save,strList
from utils.data_cleaning import HFDataCleaning
from utils.generate_features import candleCreateNP_vect_final,\
                                    generateFeatures_final
from utils.preprocessing_features_and_labels import extract_labels,\
                                                    align_features_and_labels,\
                                                    pre_processing_initial,\
                                                    pre_processing_extended,\
                                                    pre_processing
# from utils.models import make_input_fn
# from utils.models import performanceTesting,scoreFunction

### Load data etc

In [2]:
stockInfo = pd.read_csv('../utils/stockInfo.csv',header=[0,1])

In [3]:
stockInfo.dropna()[stockInfo.dropna()[('2020-07-05','exchange')]=='NMS'].sort_values(('2020-07-05','marketCap'))

date              2020-07-05                       
     ticker                  sector exchange     marketCap
7671   TCCO              Technology      NMS  5.902776e+06
2323   EDRY             Industrials      NMS  8.550177e+06
5016   MDRR             Real Estate      NMS  8.736265e+06
4262   ISIG  Communication Services      NMS  9.193270e+06
5644   NSYS              Technology      NMS  1.063012e+07
...     ...                     ...      ...           ...
4181   INTC              Technology      NMS  2.503564e+11
2733     FB  Communication Services      NMS  6.650369e+11
3462   GOOG  Communication Services      NMS  1.001645e+12
5272   MSFT              Technology      NMS  1.564160e+12
12     AAPL              Technology      NMS  1.578173e+12

[1295 rows x 4 columns]

In [4]:
# How many are there contained in the file?
print('There are',
      stockInfo.shape[0],
      'tickers contained in the file, and there is data on',
      stockInfo.isnull().sum()[1:].min(),'of them.')

There are 8850 tickers contained in the file, and there is data on 5306 of them.


## Sneak peak on the 10 largest companies

In [5]:

stockInfo.sort_values(('2020-07-05','marketCap'),ascending=False).head(10)

date              2020-07-05                       
     ticker                  sector exchange     marketCap
12     AAPL              Technology      NMS  1.578173e+12
5272   MSFT              Technology      NMS  1.564160e+12
3462   GOOG  Communication Services      NMS  1.001645e+12
2733     FB  Communication Services      NMS  6.650369e+11
699    BABA       Consumer Cyclical      NYQ  5.936536e+11
8228      V      Financial Services      NYQ  4.298068e+11
4459    JNJ              Healthcare      NYQ  3.716463e+11
8583    WMT      Consumer Defensive      NYQ  3.375968e+11
4930     MA      Financial Services      NYQ  3.035511e+11
7950    TSM              Technology      NYQ  2.837483e+11

## How is the tickers divided in sectors?

In [6]:
stockInfo.loc[:,('2020-07-05')].groupby(['sector']).count()#rename(None,axis=1)

,exchange,marketCap
sector,,
Basic Materials,137,137
Communication Services,146,146
Consumer Cyclical,318,317
Consumer Defensive,130,130
Energy,204,204
Financial,3,3
Financial Services,942,942
Healthcare,607,607
Industrials,404,404


## Lets get the X largest companies in each sector.

In [7]:
sectors = stockInfo.loc[:,('2020-07-05')].dropna(axis=0).sector.unique()

X = 5

topXsectors = pd.DataFrame(index = np.arange(X),columns = pd.MultiIndex.from_product([['Top {}'.format(X)],sectors]))

t1 = pd.DataFrame({'tickers':stockInfo.loc[:,'date'].ticker.values})
t1[stockInfo.loc[:,'2020-07-05'].columns] = stockInfo.loc[:,'2020-07-05']

for i,sector in enumerate(sectors):
    
    tempSec = t1[t1.sector==sector].sort_values('marketCap',
                                                      ascending=False).dropna(axis=0).values.T
    
    Y = len(tempSec[0][0:X]) 

    topXsectors.loc[0:(Y-1),('Top {}'.format(X),sector)] = tempSec[0][0:Y]

In [8]:
topXsectors

Top 5                                                 \
  Healthcare Basic Materials Consumer Defensive Industrials   
0        JNJ             BHP                WMT         UNP   
1        NVS             LIN                 KO          BA   
2        PFE             ECL                PEP         LMT   
3       ABBV             APD               COST         UPS   
4        ABT             SHW                FMX         RTX   

                                                                         \
  Financial Services Technology Consumer Cyclical Real Estate Utilities   
0                  V       AAPL              BABA         AMT         D   
1                 MA       MSFT              TSLA         CCI        SO   
2               PYPL        TSM                TM         PLD       AEP   
3                BAC       INTC               NKE         PSA       EXC   
4                LFC       NVDA               MCD        CSGP       SRE   

                                           
  Communication Services Energy Financial  
0                   GOOG    PTR       IIM  
1                     FB     BP       ZTR  
2                      T    SNP      OTTW  
3                    DIS    ENB       NaN  
4                    CHL    PBR       NaN

In [9]:
topXsectors.columns = topXsectors.columns.droplevel(0)

In [10]:
topXsectors

,Healthcare,Basic Materials,Consumer Defensive,Industrials,Financial Services,Technology,Consumer Cyclical,Real Estate,Utilities,Communication Services,Energy,Financial
0,JNJ,BHP,WMT,UNP,V,AAPL,BABA,AMT,D,GOOG,PTR,IIM
1,NVS,LIN,KO,BA,MA,MSFT,TSLA,CCI,SO,FB,BP,ZTR
2,PFE,ECL,PEP,LMT,PYPL,TSM,TM,PLD,AEP,T,SNP,OTTW
3,ABBV,APD,COST,UPS,BAC,INTC,NKE,PSA,EXC,DIS,ENB,NaN
4,ABT,SHW,FMX,RTX,LFC,NVDA,MCD,CSGP,SRE,CHL,PBR,NaN


In [11]:
topXsectors.columns!='Financial'

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False])

In [12]:
# List for data extraction
topXsectors.loc[:,topXsectors.columns != 'Financial'].values.flatten()

array(['JNJ', 'BHP', 'WMT', 'UNP', 'V', 'AAPL', 'BABA', 'AMT', 'D',
       'GOOG', 'PTR', 'NVS', 'LIN', 'KO', 'BA', 'MA', 'MSFT', 'TSLA',
       'CCI', 'SO', 'FB', 'BP', 'PFE', 'ECL', 'PEP', 'LMT', 'PYPL', 'TSM',
       'TM', 'PLD', 'AEP', 'T', 'SNP', 'ABBV', 'APD', 'COST', 'UPS',
       'BAC', 'INTC', 'NKE', 'PSA', 'EXC', 'DIS', 'ENB', 'ABT', 'SHW',
       'FMX', 'RTX', 'LFC', 'NVDA', 'MCD', 'CSGP', 'SRE', 'CHL', 'PBR'],
      dtype=object)

In [13]:
stockInfo.head()

date          2020-07-05                       
  ticker              sector exchange     marketCap
0      A          Healthcare      NYQ  2.738234e+10
1     AA     Basic Materials      NYQ  2.039532e+09
2   AAAU                 NaN      NaN           NaN
3   AACG  Consumer Defensive      NGM  4.157730e+07
4   AADR                 NaN      NaN           NaN

In [14]:
'IYZ' in stockInfo.loc[:,('date','ticker')].values

True

In [15]:
sectors

array(['Healthcare', 'Basic Materials', 'Consumer Defensive',
       'Industrials', 'Financial Services', 'Technology',
       'Consumer Cyclical', 'Real Estate', 'Utilities',
       'Communication Services', 'Energy', 'Financial'], dtype=object)

In [16]:
## IShares Sector ETFS

# iShares Dow Jones U.S. Basic Materials Index:     IYM
# iShares Dow Jones U.S. Consumer Goods Index:      IYK
# iShares Dow Jones U.S. Consumer Services Index:   IYC
# iShares Dow Jones U.S. Energy Index:              IYE
# iShares Dow Jones U.S. Financial Sector Index:    IYF
# iShares Dow Jones U.S. Financial Services Index:  IYG
# iShares Dow Jones U.S. Healthcare Index:          IYH
# iShares Dow Jones U.S. Industrial Index:          IYJ
# iShares Dow Jones U.S. Real Estate Index:         IYR
# iShares Dow Jones U.S. Technology Index:          IYW
# iShares Dow Jones U.S. Telecommunications Index:  IYZ
# iShares Dow Jones Transportation Average Index:   IYT
# iShares Dow Jones U.S. Utilities Index:           IDU
# iShares Cohen & Steers Realty Majors Index:       ICF

etfs = ['IYH','IYM','IYK','IYJ','IYG','IYW','IYC','IYR','IDU','IYZ','IYE','IYF']
# {i:j for i,j in zip(sectors,etfs)}
pd.DataFrame({i:j for i,j in zip(sectors,etfs)},index=[0])

,Healthcare,Basic Materials,Consumer Defensive,Industrials,Financial Services,Technology,Consumer Cyclical,Real Estate,Utilities,Communication Services,Energy,Financial
0,IYH,IYM,IYK,IYJ,IYG,IYW,IYC,IYR,IDU,IYZ,IYE,IYF


## Let's extract some data now!

In [2]:
all_tickers = ['JNJ', 'BHP', 'WMT', 'UNP', 'V', 'AAPL', 'BABA', 'AMT', 'D',
               'GOOG', 'PTR', 'NVS', 'LIN', 'KO', 'BA', 'MA', 'MSFT', 'TSLA',
               'CCI', 'SO', 'FB', 'BP', 'PFE', 'ECL', 'PEP', 'LMT', 'PYPL', 'TSM',
               'TM', 'PLD', 'AEP', 'T', 'SNP', 'ABBV', 'APD', 'COST', 'UPS',
               'BAC', 'INTC', 'NKE', 'PSA', 'EXC', 'DIS', 'ENB', 'ABT', 'SHW',
               'FMX', 'RTX', 'LFC', 'NVDA', 'MCD', 'CSGP', 'SRE', 'CHL', 'PBR',
              'IYH','IYM','IYK','IYJ','IYG','IYW','IYC','IYR','IDU','IYZ','IYE'
              ]

In [20]:
# print(os.listdir())
try:
    path = 'a:/taqhdf5'  #'a:/taqhdf5'
    os.listdir(path)
except:
    path = 'f:/taqhdf5'  #'a:/taqhdf5'
    os.listdir(path)    
# allFiles = os.listdir(path)
# print(len(allFiles), allFiles[:5], allFiles[-5:])
# print(allFiles[-10:])

#dates = np.array(['2020040' + str(i) if i < 10 else '202004' + str(i) for i in np.arange(1,16)]).astype(int)
dates = np.array(['20200501',
                  '20200504',
                  '20200505',
                  '20200506',
                  '20200507',
                  '20200508',
                  '20200511',
                  '20200512',
                  '20200513',
                  '20200514',
                  '20200515',
                  '20200518',
                  '20200519',
                  '20200520',
                  '20200521',
                  '20200522',
                  '20200526',
                  '20200527',
                  '20200528',
                  '20200529']).astype(int)#,'20200401','20200402','20200403','20200406','20200407'

# Provide a list of tickers of interest
tickers = sorted(all_tickers) # ['GOOG']#'MSFT' ['GOOG', 'MSFT'] #

# Do we need data on trades, quotes or both?
dataNeeded = 'quotes' # 'trades', 'quotes' or 'both'

extract_candles = True #False
aggHorizon = [1/6, 1/2, 1] # in minutes
extra_features_from_quotes = ['spread', 'bidsize', 'ofrsize']

data_sample = 'full'
save_output = True

exchanges = ['q','t']

# run load_data()
candles = load_data_final(dates,
                          tickers,
                          dataNeeded,
                          path,
                          verbose,
                          extract_candles,
                          aggHorizon,
                          extra_features_from_quotes,
                          data_sample,
                          exchanges,
                          save_output)

# quoteData = load_data_final(dates,
#                           tickers,
#                           dataNeeded,
#                           path,
#                           verbose,
#                           extract_candles,
#                           aggHorizon,
#                           extra_features_from_quotes,
#                           data_sample,
#                           save_output)

# if dataNeeded == 'trades':
#     tradeData = load_data(dates, tickers, dataNeeded, path, verbose)
# elif dataNeeded == 'quotes':
#     quoteData = load_data(dates, tickers, dataNeeded, path, verbose)
# elif dataNeeded == 'both':
#     tradeData, quoteData = load_data(dates, tickers, dataNeeded, path, verbose)

20 ['taqquote_20200501.h5', 'taqquote_20200504.h5', 'taqquote_20200505.h5', 'taqquote_20200506.h5', 'taqquote_20200507.h5'] ['taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
['taqquote_20200515.h5', 'taqquote_20200518.h5', 'taqquote_20200519.h5', 'taqquote_20200520.h5', 'taqquote_20200521.h5', 'taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
##### Date range #####

Date, Min: 20200501
Date, Max: 20200504


1 Lap time: 0.001

##### Data Extraction begins #####

quote data is being extracted..


2 Lap time: 0.001

### Quote Data ###

3 Lap time: 0.002
4 Lap time: 0.003
The raw H5 quote file contains:  ['QuoteIndex', 'Quotes'] 

5 Lap time: 0.737
6 Lap time: 0.746
7 Lap time: 59.099
8 Lap time: 76.259
9 Lap time: 91.182
10 Lap time: 106.068
11 Lap time: 106.449
12 Lap time: 111.140
3 Lap time: 345.773
4 Lap time: 345.773
5 Lap time: 346.587
6 Lap ti

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [4]:
# print(os.listdir())
try:
    path = 'a:/taqhdf5'  #'a:/taqhdf5'
    os.listdir(path)
except:
    path = 'f:/taqhdf5'  #'a:/taqhdf5'
    os.listdir(path)    
# allFiles = os.listdir(path)
# print(len(allFiles), allFiles[:5], allFiles[-5:])
# print(allFiles[-10:])

#dates = np.array(['2020040' + str(i) if i < 10 else '202004' + str(i) for i in np.arange(1,16)]).astype(int)
dates = np.array(['20200501',
                  '20200504',
                  '20200505',
                  '20200506',
                  '20200507',
                  '20200508',
                  '20200511',
                  '20200512',
                  '20200513',
                  '20200514',
                  '20200515',
                  '20200518',
                  '20200519',
                  '20200520',
                  '20200521',
                  '20200522',
                  '20200526',
                  '20200527',
                  '20200528',
                  '20200529']).astype(int)#,'20200401','20200402','20200403','20200406','20200407'

# Provide a list of tickers of interest
tickers = sorted(all_tickers) # ['GOOG']#'MSFT' ['GOOG', 'MSFT'] #

# Do we need data on trades, quotes or both?
dataNeeded = 'quotes' # 'trades', 'quotes' or 'both'

extract_candles = True #False
aggHorizon = [1/6, 1/2, 1] # in minutes
extra_features_from_quotes = ['spread', 'bidsize', 'ofrsize']

data_sample = 'full'
save_output = True

exchanges = ['q','t']

# run load_data()
candles = load_data_and_save(dates,
                          tickers,
                          dataNeeded,
                          path,
                          verbose,
                          extract_candles,
                          aggHorizon,
                          extra_features_from_quotes,
                          data_sample,
                          exchanges,
                          save_output)

# quoteData = load_data_final(dates,
#                           tickers,
#                           dataNeeded,
#                           path,
#                           verbose,
#                           extract_candles,
#                           aggHorizon,
#                           extra_features_from_quotes,
#                           data_sample,
#                           save_output)

# if dataNeeded == 'trades':
#     tradeData = load_data(dates, tickers, dataNeeded, path, verbose)
# elif dataNeeded == 'quotes':
#     quoteData = load_data(dates, tickers, dataNeeded, path, verbose)
# elif dataNeeded == 'both':
#     tradeData, quoteData = load_data(dates, tickers, dataNeeded, path, verbose)

20 ['taqquote_20200501.h5', 'taqquote_20200504.h5', 'taqquote_20200505.h5', 'taqquote_20200506.h5', 'taqquote_20200507.h5'] ['taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
['taqquote_20200515.h5', 'taqquote_20200518.h5', 'taqquote_20200519.h5', 'taqquote_20200520.h5', 'taqquote_20200521.h5', 'taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
##### Date range #####

Date, Min: 20200501
Date, Max: 20200529


1 Lap time: 0.001

##### Data Extraction begins #####

quote data is being extracted..


2 Lap time: 0.001

### Quote Data ###

3 Lap time: 0.081
4 Lap time: 0.081
The raw H5 quote file contains:  ['QuoteIndex', 'Quotes'] 

5 Lap time: 0.833
6 Lap time: 0.842
7 Lap time: 0.857
8 Lap time: 76.403
9 Lap time: 91.095
10 Lap time: 105.892
11 Lap time: 106.257
12 Lap time: 110.702
3 Lap time: 421.300
4 Lap time: 421.300
5 Lap time: 422.080
6 Lap tim

In [21]:
# print(os.listdir())
try:
    path = 'a:/taqhdf5'  #'a:/taqhdf5'
    os.listdir(path)
except:
    path = 'f:/taqhdf5'  #'a:/taqhdf5'
    os.listdir(path)    
# allFiles = os.listdir(path)
# print(len(allFiles), allFiles[:5], allFiles[-5:])
# print(allFiles[-10:])

#dates = np.array(['2020040' + str(i) if i < 10 else '202004' + str(i) for i in np.arange(1,16)]).astype(int)
dates = np.array(['20200505',
                  '20200506',
                  '20200507',
                  '20200508',
                  '20200511',
                  '20200512',
                  '20200513',
                  '20200514',
                  '20200515',
                  '20200518',
                  '20200519',
                  '20200520',
                  '20200521',
                  '20200522',
                  '20200526',
                  '20200527',
                  '20200528',
                  '20200529'
                  
                  
                 ]).astype(int)#,'20200401','20200402','20200403','20200406','20200407'

# Provide a list of tickers of interest
tickers = sorted(all_tickers) # ['GOOG']#'MSFT' ['GOOG', 'MSFT'] #

# Do we need data on trades, quotes or both?
dataNeeded = 'quotes' # 'trades', 'quotes' or 'both'

extract_candles = True #False
aggHorizon = [1/6, 1/2, 1] # in minutes
extra_features_from_quotes = ['spread', 'bidsize', 'ofrsize']

data_sample = 'full'
save_output = True

# run load_data()
candles = load_data_final(dates,
                          tickers,
                          dataNeeded,
                          path,
                          verbose,
                          extract_candles,
                          aggHorizon,
                          extra_features_from_quotes,
                          data_sample,
                          save_output)

dates,
                    tickers,
                    dataNeeded,
                    path,
                    verbose,
                    extract_candles = False,
                    aggHorizon = 1,
                    extra_features_from_quotes = None,
                    data_sample = 'full',
                    exchanges = ['q','t'],
                    save_output = False):

# quoteData = load_data_final(dates,
#                           tickers,
#                           dataNeeded,
#                           path,
#                           verbose,
#                           extract_candles,
#                           aggHorizon,
#                           extra_features_from_quotes,
#                           data_sample,
#                           save_output)

# if dataNeeded == 'trades':
#     tradeData = load_data(dates, tickers, dataNeeded, path, verbose)
# elif dataNeeded == 'quotes':
#     quoteData = load_data(dates, tickers, dataNeeded, path, verbose)
# elif dataNeeded == 'both':
#     tradeData, quoteData = load_data(dates, tickers, dataNeeded, path, verbose)

20 ['taqquote_20200501.h5', 'taqquote_20200504.h5', 'taqquote_20200505.h5', 'taqquote_20200506.h5', 'taqquote_20200507.h5'] ['taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
['taqquote_20200515.h5', 'taqquote_20200518.h5', 'taqquote_20200519.h5', 'taqquote_20200520.h5', 'taqquote_20200521.h5', 'taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
##### Date range #####

Date, Min: 20200505
Date, Max: 20200529


1 Lap time: 0.001

##### Data Extraction begins #####

quote data is being extracted..


2 Lap time: 0.002

### Quote Data ###

3 Lap time: 0.076
4 Lap time: 0.076
The raw H5 quote file contains:  ['QuoteIndex', 'Quotes'] 

5 Lap time: 0.868
6 Lap time: 0.878
7 Lap time: 62.353
8 Lap time: 77.898
9 Lap time: 92.528
10 Lap time: 107.148
11 Lap time: 107.512
12 Lap time: 112.094
3 Lap time: 353.492
4 Lap time: 353.493
5 Lap time: 354.299
6 Lap ti

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [ ]:
candles

In [ ]:
quoteData

In [ ]:
cleanedData = HFDataCleaning(['P1_2','p2', 'q2', 'p3'],quoteData,'quote',['q'])

In [ ]:
cleanedData

In [ ]:
cleanedData.groupby(['Ticker','Date']).count()

In [ ]:
cleanedData[cleanedData['Ticker'] == 'MSFT']

In [ ]:
candles = candleCreateNP_vect_final(data = cleanedData[cleanedData['Ticker'] == 'MSFT'],
                                    step = aggHorizon,
                                    verbose=False,
                                    fillHoles=True,
                                    sample='full',
                                    numpied=False,
                                    return_extended=extra_features_from_quotes)
candles

In [ ]:
cleanedData.loc[cleanedData['Ticker'] == 'MSFT', :]

In [ ]:
cleanedData[cleanedData['Ticker'] == 'GOOG']

In [ ]:
type([5]) == list

In [ ]:
#quoteData.shape,candles.shape
candles.shape

In [ ]:
quoteData.shape, quoteData

In [ ]:
candles

## Data Cleaning, Feature Engineering & Pre Processing

# Implemented technical features

A library: https://technical-analysis-library-in-python.readthedocs.io/en/latest/

### Features used in the literature:

* Stochastic K
* Stochastic D
* Slow Stochastic D
* Momentum/difference
* ROC
* Williams % R
* A/D Oscillator
* Disparity 5
* Disparity 10
* Price Oscillator - (detrended)
* Commodity Channel Index
* RSI

Formulas: https://papers.ssrn.com/sol3/papers.cfm?abstract_id=876544

* Moving Average
* Exponential Moving Average
* True Range - (Average)

Formulas: https://www.sciencedirect.com/science/article/pii/S0957417407001819?via%3Dihub

#### Other Technical Features
* Moving Average Convergence Divergence (MACD)

**Non-classical technical features** - **NOT IMPLEMENTED**

* Bid/Ask prices of top of book
* Spread and mid price based on top og book
* Price derivatives

Formulas: https://www.tandfonline.com/doi/full/10.1080/14697688.2015.1032546?instName=UCL+%28University+College+London%29

In [ ]:
########### Clean data ###########
    
DATA_SAMPLE = 'full' # or 'stable'

# if DATA_SAMPLE == 'stable':
#     # P1 is used for keeping data within [9.5, 16]
#     cleanedData = HFDataCleaning(['P1','p2','t1','p3'],tradeData,'trade',['q'])
# elif DATA_SAMPLE == 'full':
#     # P1_2 is used for keeping data within [9, 16.5]
#     cleanedData = HFDataCleaning(['P1_2','p2', 'q2', 'p3'],quoteData,'quote',['q'])#'t1',tradeData # q2, quotedate
    
# ########### Construct Candles ################
# # candles = candleCreateNP_vect_final(cleanedData
# #                          ,1)

# candles = candleCreateNP_vect_final(data = cleanedData,
#                                        step = 1,
#                                         verbose=False,
#                                         fillHoles=True,
#                                         sample='stable',
#                                         numpied=True)

########### Generate Features ################

n_feature_lags = 1
features = generateFeatures_final_test(candles, 
                                       listOfFeatures = [
                                           'pastobs',
                                            'spread',
                                           'bidsize',
                                           'ofrsize',
                                            'stok',
                                            'stod',
                                            'sstod',
                                            'wilr',
                                            'roc',
                                            'rsi',
                                            'atr',
                                            'cci',
                                            'dpo',
                                            'sma',
                                            'ema',
                                            'macd',
                                            'dis5',
                                            'dis10',
                                           ], 
                                    feature_lags = n_feature_lags)

########### Generate Labels ################

n_classes = 3

labels = extract_labels(data = candles['price'].values, classes = n_classes, group_style = 'equal')

########### Align Data ################

# from imported function (see testing_preprocessing_features_and_labels.ipynb for thorough experimenting with all the cut-offs):    
X, y = align_features_and_labels(candles = candles['price'].values, 
                                 prediction_horizon = 1, 
                                 features = features, 
                                 n_feature_lags = n_feature_lags, 
                                 n_classes = n_classes, # 5,
                                 safe_burn_in = False, 
                                 data_sample = 'full')

In [ ]:
candles['price'].values

In [ ]:
features

In [ ]:
X

In [ ]:
quoteData

In [ ]:
candles[-10:]

In [ ]:
t1 = pd.DataFrame({'0':np.random.randint(0,10,1000000),
                   '1':np.random.randint(0,10,1000000),
                   '2':np.random.randint(0,10,1000000)})

# t2 = pd.Series({'0':np.random.randint(0,10,10)})

In [ ]:
#t1.loc[:,['0','1','2']] = t1.loc[:,['0','1','2']] - t1.loc[:,'2']

In [ ]:
t1.loc[:,['0','1','2']] - t1.loc[:,'2']

In [ ]:
type(t1.loc[:,'2'])

In [ ]:
t1[['0','1','2']]

In [ ]:
%timeit t1[['0','1','2']].values-t1[['2']].values

In [ ]:
%timeit t1[['0','1','2']].subtract(t1.loc[:,'2'],axis=0)

In [ ]:
t1

## Let's investigate the features a little bit

In [ ]:
X.describe()

## Standardization, Normalization (MinMax), Norm-Scaling, Quantile and Power Transformation

**Inspiration:**

* [ScikitLearn Overview](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#sphx-glr-auto-examples-preprocessing-plot-all-scaling-py)
* [A Note on Feature Scaling and Normalization](http://sebastianraschka.com/Articles/2014_about_feature_scaling.html)

In [ ]:
# Setting up the Scalers!
mm_scaler = MinMaxScaler()
scaler = StandardScaler()
norm_scaler = Normalizer()
pt = PowerTransformer()
ptNst = PowerTransformer(standardize=False)
qtUni = QuantileTransformer(n_quantiles=100)
qtGau = QuantileTransformer(n_quantiles=100,output_distribution='normal')
robo = RobustScaler()


## Split data into train and test set

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train = X_train.reset_index(drop=True) 
X_test = X_test.reset_index(drop=True)
# y_train = y_train.reset_index(drop=True)
# y_test = y_test.reset_index(drop=True)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# featurePreProcessing = {'open':'std',
#                         'close':'std',
#                         'high':'std',
#                         'low':'std',
                        
#                         'stok':'std',
#                         'stod':'std',
#                         'sstod':'std',
#                         'wilr':'std',
#                         'ema':'std',
#                         'sma':'std',
#                         'dis5':'sub',
#                         'dis10':'sub',
#                         'macd_diff':'act',
#                         'roc':'actde',
#                         'atr':'actde',
#                         'rsi':'std',
#                         'cci':'quantgau',
#                         'dpo':'quantgau',
#                         'macd':'quantgau',
#                         'macd_signal':'quantgau'}
featurePreProcessing = {col:'quantgau' for col in X.columns}
ppX_train,ppX_test = pre_processing(X_train,
                                       X_test,
                                       featurePreProcessing,
                                       100)

In [ ]:
corr = ppX_train.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
X_train[['open','high']].corr()

## Feature Selection

In [ ]:
start = time.time()

## Setting up the model and corresponding parameters

rf = ensemble.RandomForestClassifier()
param_grid = {'knn': {'n_neighbors': [1,3,5,7,9,11,13,15,17,19]},
                  'rf': {'n_estimators': [50,100,200], 'max_features': ['auto', None], 'min_samples_leaf': [1, 5, 10]}}

## Setting parameters for grid search
cv_folds = 5
n_jobs = 1

## Performing grid search
grid_search = GridSearchCV(rf, param_grid['rf'], cv = cv_folds, n_jobs = n_jobs)
grid_search.fit(ppX_train,y_train)

# store the best hyperparameters and initialize a separate random forest with those parameters
rf_params = grid_search.best_params_
rf = ensemble.RandomForestClassifier(**rf_params)

# refit the random forest using only the correctly accessible training data
# and return feature importances. This is not the perfect solution but it is 
# better than extracting feature importances from the grid search above which 
# is refit on all data (also test set) of the inner folds

rf.fit(ppX_train, y_train)
rf_features =  rf.feature_importances_




In [ ]:
# compute mean of importances across folds and select those above the mean
# rf_features = np.mean(rf_features, axis=0)            
threshold_value = np.mean(rf_features) 
rf_features_w = np.where(rf_features > threshold_value)[0]

end = time.time()

if verbose:
    print("RF Feature selection final best features: " + str(rf_features_w))
    print('The model selection took %.3f seconds.' % (end-start))

In [ ]:
# reduce data sets using selected features X_train = X_train[:, rf_features]
msX_test,msX_train = ppX_test.iloc[:, rf_features_w],ppX_train.iloc[:, rf_features_w]

# Refitting

rf.fit(msX_train,y_train)

# Evaluating
rf.score(msX_test,y_test)

# store selected features
# total_features[counter, :] = str(rf_features)

In [ ]:
corr = msX_train.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

## Logistic Regression in Tensorflow

In [ ]:
#### Building a Logistic Regression Model in Tensorflow
msX_train = ppX_train.copy(deep=True)
msX_test = ppX_test.copy(deep=True)
## Setting up data
NUMERIC_COLUMNS = msX_train.columns

feature_columns = []

for feature_name in NUMERIC_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))
    
## Arange data correctly
train_input_fn = make_input_fn(msX_train, y_train.astype(int))
eval_input_fn = make_input_fn(msX_test, y_test.astype(int), num_epochs=1, shuffle=False)

## Inspecting the data
ds = make_input_fn(msX_train, y_train.astype(int), batch_size=10)()
for feature_batch, label_batch in ds.take(1):
    print('Some feature keys:', list(feature_batch.keys()))
    print()
    print('A batch of class:', feature_batch['sstod'].numpy())
    print()
    print('A batch of Labels:', label_batch.numpy())
    
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns,n_classes=3)
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()
print(result)

In [ ]:
pred_dicts = list(linear_est.predict(eval_input_fn))
print(len(pred_dicts))
pred_dicts[0]
# probs = pd.Series([pred['probabilities'][1] for pred in pred_dicts])

# probs.plot(kind='hist', bins=20, title='predicted probabilities')

## Lets test some performance

### Lets test different preprocessing methods

In [ ]:
for ppm in ['std','quantgau','quantuni','pow','minmax']:#'act','actde',
    print('The preprocessing method tested is: %s\n' % ppm)
    testDict = {col:ppm for col in X.columns}
    performanceTesting(X,y,5,2020,testDict,verbose=0)
    print('\n')

## Pre-Process features individually

In [ ]:
featurePreProcessing = {'open':'std',
                        'close':'std',
                        'high':'std',
                        'low':'std',
                        'stok':'std',
                        'stod':'std',
                        'sstod':'std',
                        'wilr':'std',
                        'ema':'std',
                        'sma':'std',
                        'dis5':'sub',
                        'dis10':'sub',
                        'macd_diff':'act',
                        'roc':'actde',
                        'atr':'actde',
                        'rsi':'std',
                        'cci':'quantgau',
                        'dpo':'quantgau',
                        'macd':'quantgau',
                        'macd_signal':'quantgau'}
performanceTesting(X,y,5,2020,featurePreProcessing,verbose=0)